In [2]:
import datetime
from ib_insync import *
import pandas as pd
from pandas import Timestamp
from pytvlwcharts import *
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager import StrategyManager
    from strategy_manager.strategies.temple_w_fill import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager import StrategyManager
    from strategy_manager.strategies.temple_w_fill import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

In [3]:
ib = connect_to_IB()
trade_manager = TradeManager()

In [23]:
# Functions
   
def trade(ib, contract, quantity, order_type='MKT', urgency='Patient', orderRef="", limit=None, outsideRth=True):
    """
    Place an Order on the exchange via ib_insync.
    
    :param ib: ib insync instance.
    :param contract: ib.Contract
    :param quantity: order size as a signed integer (quantity > 0 means 'BUY' and quantity < 0 means 'SELL')
    :param order_type: order type such as 'LMT', 'MKT' etc.
    :param urgency: 'Patient' (default), 'Normal', 'Urgent'
    :param limit: if order_type 'LMT' state limit as float
    """
    ib.qualifyContracts(contract)

    # Create order object
    action = 'BUY' if quantity > 0 else 'SELL'
    totalQuantity = int(abs(quantity))

    if order_type == 'LMT':
        assert limit, "Limit price must be specified for limit orders."
        lmtPrice = float(limit)
        order = LimitOrder(action, totalQuantity, lmtPrice)
    elif order_type == 'MKT':
        order = MarketOrder(action, totalQuantity)

    # order.algoStrategy = 'Adaptive'
    # if urgency == 'Normal':
    #     order.algoParams = [TagValue('adaptivePriority', 'Normal')]
    # elif urgency == 'Urgent':
    #     order.algoParams = [TagValue('adaptivePriority', 'Urgent')]
    # else:
    #     order.algoParams = [TagValue('adaptivePriority', 'Patient')]

    order.orderRef = orderRef
    order.outsideRth = outsideRth
    # Place the order
    trade = ib.placeOrder(contract, order)
    ib.sleep(1)
    return trade
       

def on_fill( trade, fill):
    # Handle fill event
    message_dict = {
        'type': 'fill',
        'strategy': 'my_symbol',
        'trade': trade,
        'fill': fill
    }
    return message_dict
    




In [24]:
# TRADE LOGIC

# Add Trading logic
contract = Stock('AAPL', 'SMART', 'USD')
trade = trade(ib,contract,1,order_type='LMT',orderRef='JUPYTER', limit=99)

    # Assign callbacks for order updates
trade.fillEvent += on_fill

ib.sleep(1)
        
while True:
    # This integrates the ib_insync event loop
    # Additional strategy logic here
    ib.sleep(1)
    if trade.order and trade.orderStatus.status != "Cancelled":
        #trade = ib.cancelOrder(trade.order)
        print(trade.orderStatus.status)
    
    if trade.orderStatus.status == 'Submitted':
        print(trade)
        break
    


Submitted
Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=165, permId=1365223704, action='BUY', totalQuantity=1.0, lmtPrice=99.0, auxPrice=0.0, orderRef='JUPYTER', outsideRth=True), orderStatus=OrderStatus(orderId=165, status='Submitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=1365223704, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 4, 5, 20, 33, 6, 547111, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 4, 5, 20, 33, 6, 963118, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0)], advancedError='')


In [10]:
Strategy = Strategy()

TypeError: Strategy.__init__() missing 2 required positional arguments: 'client_id' and 'strategy_manager'

In [ ]:
StrategyManager = StrategyManager()